## Identification of interesting VIIRS files

This notebook process a general bash file that retrieve VIIRS images to return a new bash file with the images that mathches ICESat-2 data in a given temporal window

In [4]:
import pandas as pd
from icepyx import icesat2data as ipd

In [5]:
# path with VIIRS's retrieval files
path_viirs_retrieve = "data/VIIRS_bash/"
# name of the original bash file to retrieve the VIIRS data
exe_old = "9531467636-download.sh"
# name of the output bash file 
exe_new = "VIIRS-dt10-download.sh"

In [10]:
# Spatial extend for the ICESat-2 search 
# Observation: in case that the VIIRS file retrieve images in a different spatial extent, then we are no going to have matches, 
# independently of the window. 
spatial_extent = [-50, 70, -49, 71]

# Temporal window in minutes
hr = 10 / 60 
# Maximim number of files we are interested in retrieve 
max_viirs = 3

In [ ]:
file = open(path_viirs_retrieve + exe_old, 'r')
new_file = open(path_viirs_retrieve + exe_new, 'x')

viirs_names = []

counter = 0
counter_viirs = 0

for line in file:
    
    #print(line)
    
    if ("https://ladsweb.modaps.eosdis.nasa.gov" in line) and (len(line) == 146):
        
        if len(viirs_names) < max_viirs:

            Vfile = line[:-1]

            f_t = int(Vfile[-33:-26]+Vfile[-25:-21])
            f_t = pd.to_datetime(f_t, format="%Y%j%H%M")

            # Temporal search window for 
            start = f_t - pd.DateOffset(hours=hr)
            end   = f_t + pd.DateOffset(hours=hr)

            start_date_str = start.strftime('%Y-%m-%d')
            end_date_str   = end.strftime('%Y-%m-%d')
            start_time_str = start.strftime('%H:%M:%S')
            end_time_str   = end.strftime('%H:%M:%S')

            try:

                region_a = ipd.Icesat2Data("ATL06", spatial_extent, [start_date_str, end_date_str], start_time_str, end_time_str)
                avail_granules = region_a.avail_granules(ids=True)

                viirs_names.append(Vfile)
                new_file.write(Vfile + "\n") # add \n
                assert len(avail_granules) > 0
                counter_viirs += 1

            except AssertionError:
                None
                
            counter += 1
        
        if counter % 10 == 0:
            print(">>> There are a total of", counter_viirs, "viirs files founded out of", counter, "file names.")
        
    else:
        
        new_file.write(line)
        
    
new_file.close()

>>> There are a total of 0 viirs files founded out of 10 file names.
>>> There are a total of 0 viirs files founded out of 20 file names.
>>> There are a total of 0 viirs files founded out of 30 file names.
>>> There are a total of 0 viirs files founded out of 40 file names.
>>> There are a total of 0 viirs files founded out of 50 file names.
>>> There are a total of 0 viirs files founded out of 60 file names.
>>> There are a total of 0 viirs files founded out of 70 file names.
>>> There are a total of 0 viirs files founded out of 80 file names.
>>> There are a total of 0 viirs files founded out of 90 file names.
>>> There are a total of 0 viirs files founded out of 100 file names.
>>> There are a total of 0 viirs files founded out of 110 file names.
>>> There are a total of 0 viirs files founded out of 120 file names.
>>> There are a total of 0 viirs files founded out of 130 file names.


In [3]:
100 % 11

1